In [7]:
import pandas as pd
import numpy as np
import os
import string
import re
from sklearn import decomposition
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

C:\Users\tidyq\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\tidyq\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [36]:
curr_dir= os.getcwd()
demographics = pd.read_csv(curr_dir+"//train/demographics.csv")
event_calendar = pd.read_csv(curr_dir+"//train/event_calendar.csv")
historical_volume = pd.read_csv(curr_dir+"//train/historical_volume.csv")
industry_soda_sales = pd.read_csv(curr_dir+"//train/industry_soda_sales.csv")
industry_volume = pd.read_csv(curr_dir+"//train/industry_volume.csv")
price_sales_promotion = pd.read_csv(curr_dir+"//train/price_sales_promotion.csv")
weather = pd.read_csv(curr_dir+"//train/weather.csv")

In [41]:
historical_volume.head()

,Agency,SKU,YearMonth,Volume
0,22,01,201301,52.2720
1,22,02,201301,110.7000
2,58,23,201301,0.0000
3,48,07,201301,28.3200
4,22,05,201301,238.5387


In [38]:
vol_test = pd.read_csv(curr_dir+"//test/volume_forecast.csv")
vol_test1 = pd.read_csv(curr_dir+"//test/volume_forecast.csv")

In [39]:
def cat_split(row):
    try:
        text = row
        txt1, txt2 = text.split('_')
        return txt2
    except:
        return np.nan

historical_volume['Agency'] = historical_volume.Agency.apply(lambda val: cat_split(val))
historical_volume['SKU'] = historical_volume.SKU.apply(lambda val: cat_split(val))

In [40]:
def cat_split(row):
    try:
        text = row
        txt1, txt2 = text.split('_')
        return txt2
    except:
        return np.nan

vol_test['Agency'] = vol_test.Agency.apply(lambda val: cat_split(val))
vol_test['SKU'] = vol_test.SKU.apply(lambda val: cat_split(val))

In [11]:
historical_volume = pd.get_dummies(historical_volume, columns=["SKU"])
historical_volume= pd.get_dummies(historical_volume, columns=["Agency"])

In [12]:
vol_test = pd.get_dummies(vol_test, columns=["SKU"])
vol_test = pd.get_dummies(vol_test, columns=["Agency"])

In [42]:
vol_test.head()

,Agency,SKU,Volume
0,23,06,NaN
1,34,20,NaN
2,24,02,NaN
3,40,06,NaN
4,42,32,NaN


In [43]:
vol_test.drop(['Volume'],axis=1,inplace = True)

In [44]:
historical_volume.drop(['YearMonth'],axis=1,inplace = True)

In [45]:
train = historical_volume.copy()

In [55]:
train.head()

,Agency,SKU,Volume
0,22,01,52.2720
1,22,02,110.7000
2,58,23,0.0000
3,48,07,28.3200
4,22,05,238.5387


In [47]:
target = 'Volume'

In [57]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='rmse', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Disbursed'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain['Disbursed'].values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Disbursed'], dtrain_predprob))
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [59]:
#Choose all predictors except target & IDcols
predictors = [x for x in train.columns if x not in [target]]
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'reg:linear',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

modelfit(xgb1, train, predictors)

KeyError: 'Disbursed'